In [1]:
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

import django
django.setup()

In [2]:
from data.models import HourData

In [3]:
import requests
from datetime import timedelta, datetime

In [4]:
up_dn_list = []
last_data = []
dn_list = []
up_list = []
new_dn_list = []
new_up_list = []
other_up_dn_list = []

continue_up_down = "D0"
prev_up_down = "S"
work_1_0_1 = "S"
work_1_5_1 = "S"
work_1_0_2 = "S"
work_1_5_2 = "S"
last_time_1_0_1 = datetime.now()
last_time_1_5_1 = datetime.now()
last_time_1_0_2 = datetime.now()
last_time_1_5_2 = datetime.now()

last_dn, last_up = "", ""

max_price, min_price = 0, 0
work_up_high, work_dn_low = 1000000, 0
volume = 0
prev_low = 0
prev_high = 100000
start = 0
last_up = 0
last_dn = 0
volume_up_dn = ""
hour_up_down = ""
prev_signal_up_price = 0
prev_signal_down_price = 0

In [5]:

while True:
    URL = "https://www.bitmex.com/api/v1/trade/bucketed?symbol=XBT&binSize=1h&partial=false&start={}&count=1000&reverse=false".format(start)
    req = requests.get(URL).json()
    for data in req:
        volume_up_dn = ""
        signal = ""
        signal_price = 0
        hour_up_down = ""
        now_work_1_0_1 = ""
        now_work_1_5_1 = ""
        now_work_1_0_2 = ""
        now_work_1_5_2 = ""
        datetime = datetime.strptime(data['timestamp'].replace("T", " ")[0:19], "%Y-%m-%d %H:%M:%S") + timedelta(hours=8)
        open_price = data['open'] if data['open'] != None else 0
        high_price = data['high'] if data['high'] != None else 0
        low_price = data['low'] if data['low'] != None else 0
        close_price = data['close'] if data['close'] != None else 0
        avg_price = (high_price + low_price) / 2
        
        if volume != 0:
            volume_rate = data['volume'] / volume
        else:
            volume_rate = 0
            
        volume = data['volume']

        flag = False

        if open_price > close_price:
            UD = "D"

        elif open_price < close_price:
            UD = "U"

        else:
            UD = "S"

        if len(last_data) > 3:
            last_data.pop(0)
            volume_bigger_count = 0
            volume_smaller_count = 0

            for i in last_data:
                if i[2] < volume:
                    volume_bigger_count += 1

                if i[2] > volume:
                    volume_smaller_count += 1

            if volume_bigger_count == 3:
                volume_up_dn = "UP"

            if volume_smaller_count == 3:
                volume_up_dn = "DN"

        last_data.append([low_price, high_price, volume])
        
        if continue_up_down[0] == "U":
            if UD == "U":
                continue_up_down = "U" + str(int(continue_up_down[1:]) + 1)
            elif UD == "D":
                continue_up_down = "D1"
        else:
            if UD == "U":
                continue_up_down = "U1"
            elif UD == "D":
                continue_up_down = "D" + str(int(continue_up_down[1:]) + 1)
        
        if len(up_dn_list) > 3:
            up_dn_list.pop(0)
        up_dn_list.append(UD)
        
        if up_dn_list.count("U") > 2:
            if high_price >= last_up:
                hour_up_down = "UP(U)"
            else:
                hour_up_down = "UP(D)"
            last_up = high_price
        
        if up_dn_list.count("D") > 2:
            if low_price >= last_dn:
                hour_up_down = "DN(U)"
            else:
                hour_up_down = "DN(D)"
            last_dn = low_price
            
        if UD == "U":
            if other_up_dn_list.count("D") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                for idx, element in enumerate(other_up_dn_list):
                    if element == "D":
                        count += 1
                        max_price_list.append(dn_list[idx])
                    else:
                        if count >= 3:
                            break
                    min_price_list.append(up_list[idx])

                if prev_up_down == "D" and max(max_price_list) == max_price_list[len(max_price_list) - 1]:
                    if prev_signal_down_price != 0:
                        if prev_signal_down_price < min(min_price_list):
                            signal = "fD(U)"
                        elif prev_signal_down_price == min(min_price_list):
                            signal = "fD(0)"
                        else:
                            signal = "fD(D)"
                    prev_signal_down_price = min(min_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()

        if UD == "D":
            if other_up_dn_list.count("U") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_D = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "U":
                        count += 1
                        min_price_list.append(up_list[idx])
                    else:
                        if count >= 3:
                            break
                    max_price_list.append(dn_list[idx])

                if prev_up_down == "U" and min(min_price_list) == min_price_list[len(min_price_list) - 1]:
                    print(datetime)
                    print(other_up_dn_list)
                    print(max_price_list, min_price_list)
                    if prev_signal_down_price != 0:
                        if prev_signal_up_price < max(max_price_list):
                            signal = "fU(U)"
                        elif prev_signal_up_price == max(max_price_list):
                            signal = "fU(0)"
                        else:
                            signal = "fU(D)"
                    prev_signal_up_price = max(max_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
        
        if len(dn_list) > 5:
            dn_list.pop(0)
        dn_list.append(high_price)
            
        if len(up_list) > 5:
            up_list.pop(0)
        up_list.append(low_price)
        
        if len(other_up_dn_list) > 5:
            other_up_dn_list.pop(0)
        other_up_dn_list.append(UD)
        
        if volume_up_dn == "UP":
            if (max_price < high_price) and (volume_rate > 1):
                if work_1_0_1[0] == "U":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0_1 = True
                    else:
                        work_1_0_1 = "U" + str(int(work_1_0_1[1:]) + 1)
                else:
                    work_1_0_1 = "U1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1):
                if work_1_0_1[0] == "D":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0 = True
                    else:
                        work_1_0_1 = "D" + str(int(work_1_0_1[1:]) + 1)
                
                else:
                    work_1_0_1 = "D1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
                
            if (max_price < high_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "U":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        continue_flag_1_5_1 = True
                    else:
                        work_1_5_1 = "U" + str(int(work_1_5_1[1:]) + 1)
                else:
                    work_1_5_1 = "U1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_1 = "D" + str(int(work_1_5_1[1:]) + 1)
                
                else:
                    work_1_5_1 = "D1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime

            if (max_price < high_price and min_price < low_price) and (volume_rate > 1):
                if work_1_0_2[0] == "U":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "U" + str(int(work_1_0_2[1:]) + 1)
                else:
                    work_1_0_2 = "U1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1):
                if work_1_0_2[0] == "D":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "D" + str(int(work_1_0_2[1:]) + 1)
                
                else:
                    work_1_0_2 = "D1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime     
                
                
            if (max_price < high_price and min_price < low_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "U":
                    if datetime == last_time_1_5_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "U" + str(int(work_1_5_2[1:]) + 1)
                else:
                    work_1_5_2 = "U1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "D" + str(int(work_1_5_2[1:]) + 1)
                
                else:
                    work_1_5_2 = "D1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime

            max_price, min_price = high_price, low_price
            
        if now_work_1_0_1 != "":
            now_work_1_0_1 = "w" + now_work_1_0_1
        
        if now_work_1_5_1 != "":
            now_work_1_5_1 = "w" + now_work_1_5_1
            
        if now_work_1_0_2 != "":
            now_work_1_0_2 = "w" + now_work_1_0_2
            
        if now_work_1_5_2 != "":
            now_work_1_5_2 = "w" + now_work_1_5_2
        
        print(hour_up_down)
        element = HourData()
        element.datetime = datetime
        element.open_price = open_price
        element.min_price = low_price
        element.max_price = high_price
        element.close_price = close_price
        element.avg_price = avg_price
        element.volume = volume
        element.volume_rate = volume_rate
        element.volume_up_dn = volume_up_dn
        element.up_down = UD
        element.continue_up_down = continue_up_down
        element.hour_up_down = hour_up_down
        element.signal = signal
        element.signal_price = signal_price
        element.work_two_1_0 = now_work_1_0_1
        element.work_two_1_5 = now_work_1_5_1
        element.work_one_1_0 = now_work_1_0_2
        element.work_one_1_5 = now_work_1_5_2
        element.save()
        
        prev_up_down = UD
        
    start += 1000




DN(U)




DN(D)
DN(D)
DN(U)

UP(U)
2015-09-26 10:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[235.29, 235.36, 235.2] [235.29, 235.19, 234.9]
UP(U)








DN(D)
DN(D)


UP(D)
2015-09-27 00:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[235, 234.87, 233.85] [235, 234.1, 232.91]

DN(U)


UP(D)
UP(D)
2015-09-27 06:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[234.67, 234.73, 234, 233.8] [234.32, 234.05, 233.69, 233.68]
UP(D)
UP(D)












DN(D)
DN(U)
DN(D)





UP(D)


DN(U)
DN(D)
DN(D)
DN(U)

UP(U)
UP(D)
UP(D)
2015-09-28 15:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[237.8, 237.82, 238.23, 234.58, 233.93] [237.37, 237.34, 234.45, 233.6, 233.39]
UP(D)
UP(U)


UP(U)
2015-09-28 21:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[239.16, 238.6, 238.7] [238.57, 238.31, 237.3]

UP(D)
UP(U)
UP(D)
2015-09-29 01:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[239.31, 239.7, 238.19] [238.8, 238.15, 237.87]
UP(D)


2015-09-29 04:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[239, 239.31, 239.7] [239, 238.8, 238.15]
DN(U)






UP(U)

UP(U)
UP(U)



D

DN(U)


DN(U)

2015-10-19 07:00:00
['U', 'D', 'D', 'U', 'D', 'U']
[265.73, 266.9, 265.21] [264.65, 264.15, 263.25]
DN(U)

UP(D)
UP(U)
2015-10-19 11:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[267.73, 266.99, 265.69] [266.38, 265.68, 263.96]
UP(U)
UP(D)
2015-10-19 13:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[267.49, 267.73, 266.99] [266.65, 266.38, 265.68]

DN(D)
DN(D)
DN(U)




DN(D)


DN(U)


2015-10-20 03:00:00
['U', 'U', 'D', 'D', 'U', 'D']
[266.04, 266.12, 266.98] [265.25, 265.09, 263.33]

UP(D)

DN(D)

DN(U)

UP(U)
2015-10-20 11:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[266.44, 266.95, 266.37] [265.82, 265.39, 263.63]


DN(D)
DN(U)

UP(U)
2015-10-20 17:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[268.46, 267.9, 265.37] [265.9, 265.6, 264.78]
UP(U)



2015-10-20 21:00:00
['U', 'U', 'D', 'D', 'U', 'U']
[267.5, 267.2, 268.46] [267.3, 266.25, 265.9]

UP(U)
UP(U)
2015-10-21 00:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[269.53, 268.91, 267.5] [268.64, 267.45, 267.3]






DN(U)
DN(U)

UP(U)
UP(U)
2015-10-21 11:0

2015-11-05 12:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[463.5, 450, 433.16] [444.27, 409.2, 390]


DN(D)
DN(U)
DN(U)
DN(D)
DN(U)
DN(U)
DN(U)

UP(D)
2015-11-05 23:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[415.7, 406.57, 408] [396.59, 386.4, 384.4]



2015-11-06 02:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[409, 415.7, 406.57] [393.21, 396.59, 386.4]
DN(U)

UP(U)
UP(D)
UP(D)
2015-11-06 07:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[428.87, 429.99, 432.14, 414.22] [425, 418.55, 408.98, 403.06]
UP(U)

DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(U)

UP(D)
2015-11-06 17:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[384.91, 378, 371.85] [370.53, 361.83, 359]

UP(D)
UP(D)
UP(U)
UP(U)
2015-11-06 22:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[382.51, 377.76, 368.61, 370.51] [375.64, 366.66, 363.71, 361.57]
UP(D)



UP(U)
2015-11-07 03:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[403.88, 408.32, 388.86] [391, 383.37, 375.96]
UP(D)


DN(U)



2015-11-07 10:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[386.79, 387.59, 375.89] [373.91, 370.01, 364.14]


201



UP(D)
2015-11-26 11:00:00
['U', 'U', 'U', 'S', 'D', 'U']
[334.02, 333, 330.09] [331.75, 329.25, 328.85]
UP(U)
UP(U)
UP(U)
2015-11-26 14:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[335.5, 334.64, 334.02] [333.25, 332.71, 331.75]

UP(D)
UP(U)
UP(U)
2015-11-26 18:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[345, 338.88, 334.6] [338.25, 335.16, 333.6]
UP(U)
UP(U)
UP(U)
UP(U)
UP(D)
2015-11-26 23:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[360.77, 372.42, 353.29, 349.09] [352.7, 352.6, 346.03, 341.63]
UP(D)

DN(U)
DN(U)


UP(D)
UP(U)
UP(D)
UP(U)
2015-11-27 09:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[359.73, 354.98, 355, 353.01] [353.26, 351.2, 349.26, 347.02]
UP(D)
UP(U)

DN(U)

DN(U)



UP(U)
UP(U)
2015-11-27 20:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[367.4, 366.98, 365.39] [362.82, 359.79, 352.49]
UP(D)
UP(D)
UP(U)

UP(D)
UP(U)

UP(D)
2015-11-28 04:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[363.47, 364.5, 359.68] [362.07, 357, 355]

DN(U)





DN(D)
DN(D)









DN(U)
DN(U)

UP(D)
2015-11-29 02:00:00
['U', 'U', 'U

DN(D)
DN(U)
DN(D)
DN(D)

UP(D)
2015-12-14 12:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[441.59, 441.9, 443] [438.5, 439.73, 432.48]
UP(U)
UP(D)
UP(U)
UP(U)
2015-12-14 16:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[444.7, 443, 442.24] [441.7, 442.22, 439.92]
UP(D)
UP(U)






2015-12-15 00:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[450.75, 450.99, 444.84] [447.89, 443.15, 440.15]


2015-12-15 02:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[453.09, 450.75, 450.99] [446.34, 447.89, 443.15]

DN(U)
DN(D)
DN(D)
DN(U)
DN(D)

UP(U)
UP(U)
UP(U)
2015-12-15 12:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[462.32, 454.38, 453.59, 449.05, 448.69] [451.84, 450.1, 447.48, 446.03, 445.79]
UP(U)

DN(U)
DN(D)

2015-12-15 17:00:00
['U', 'U', 'D', 'D', 'D', 'U']
[462, 461.6, 462.32] [459.09, 455.68, 451.84]


DN(U)
DN(D)
DN(U)


UP(D)
UP(U)
2015-12-16 02:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[468.89, 464.33, 461] [463.4, 460.14, 457.59]

UP(D)
UP(U)
2015-12-16 05:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[467.01, 465.95, 468.89] [464.13, 463

...


DN(D)

DN(U)

2021-06-16 16:00:00
['U', 'D', 'U', 'D', 'D', 'U']
[40472, 40399, 40070] [40172, 39860, 39585.5]

DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)
DN(U)
DN(D)
DN(D)
DN(U)
DN(D)


UP(D)
UP(U)
2021-06-17 13:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[38928, 38829, 38778, 38626] [38670, 38609, 38520.5, 38190.5]
UP(U)


2021-06-17 16:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[39266.5, 38928, 38829] [38714.5, 38670, 38609]
DN(U)

UP(U)
2021-06-17 19:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[39540, 39408.5, 39266.5] [39128.5, 39072, 38714.5]


DN(D)
DN(U)
DN(D)




DN(D)



UP(D)
2021-06-18 09:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[38144, 37925, 37817] [37813, 37599, 37317.5]






DN(D)
DN(U)


UP(D)

DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)


UP(D)
2021-06-19 07:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[35685, 35819.5, 35549.5] [35380, 35191.5, 35184]
UP(D)
UP(U)
2021-06-19 09:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[35891.5, 35685, 35819.5] [35386, 35380, 35191.5]


2021-06-19 1

UP(D)
UP(U)
UP(U)
UP(D)
UP(U)
2021-07-06 07:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[34185.5, 34304, 34200.5] [33827.5, 33976, 33686]

DN(D)

DN(U)



UP(U)
2021-07-06 15:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[34850, 34777, 33929] [34620.5, 33909.5, 33715]

UP(U)
UP(U)
2021-07-06 18:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[35101, 34934.5, 34850] [34730, 34691, 34620.5]






DN(D)
DN(U)
DN(D)
DN(U)
DN(D)
DN(U)
DN(D)



UP(D)
UP(U)
UP(U)
UP(U)
UP(U)
2021-07-07 15:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[34924.5, 34799, 34553, 34283, 34243] [34617.5, 34400.5, 34215, 34017.5, 34001.5]
UP(D)
UP(U)
UP(U)



2021-07-07 21:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[35031, 34993, 34937] [34602.5, 34712, 34586]
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)

UP(D)
2021-07-08 04:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[34769, 34682, 34601] [34564, 34462, 34425]
UP(D)

DN(D)
DN(U)
DN(D)
DN(D)
DN(U)


UP(D)
2021-07-08 14:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[33350, 33370.5, 33385] [33150, 33249, 33144.5]




UP(D)

UP(U)

DN(D)


U




UP(U)
UP(U)
UP(U)
UP(U)
2021-07-26 11:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[39908, 36887.5, 35433, 34660, 34562.5, 34458] [36712.5, 35266.5, 34602, 34469.5, 34433, 34320.5]
UP(D)
UP(D)
UP(U)
UP(U)
UP(U)
2021-07-26 16:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[39290, 38670.5, 38579, 38380] [38415.5, 38246, 38152.5, 38020]
UP(D)
UP(D)
UP(U)
UP(U)
2021-07-26 20:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[38618, 38392, 38323] [38130, 38020.5, 37720]
UP(U)
UP(U)
2021-07-26 22:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[38931, 38618, 38392] [38236.5, 38130, 38020.5]

DN(U)


UP(U)
UP(U)
2021-07-27 04:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[40000, 39374.5, 39454, 38635] [39175, 38760, 38482, 38216]
UP(U)

DN(D)
DN(D)
DN(U)


DN(D)


UP(D)
2021-07-27 15:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[37207, 37088.5, 36934] [36883, 36705, 36379]
UP(U)
UP(D)
UP(U)
UP(U)
UP(U)
UP(U)
UP(U)
UP(U)
2021-07-27 23:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[38749.5, 38459, 38043.5, 37721.5, 37486, 37472] [38220, 37717.5, 37549.5, 372




UP(U)
2021-08-15 10:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[47262.5, 47202, 47091.5] [46941, 46870, 46626]

UP(U)
2021-08-15 12:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[47350.5, 47262.5, 47202] [47160, 46941, 46870]

DN(U)
DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(U)







UP(D)
UP(U)
UP(U)
UP(U)
UP(U)
2021-08-16 08:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[47263.5, 46990, 46724.5, 46431.5, 46305.5, 46050] [46875.5, 46500, 46266, 46098, 45854, 45620.5]
UP(U)
UP(U)
UP(D)
UP(U)
2021-08-16 12:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[48023, 47744, 47865] [47609.5, 47535.5, 46810]
UP(U)
UP(D)
UP(U)


DN(U)
DN(U)

UP(U)
2021-08-16 21:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[47680, 47579, 47243] [47421.5, 47199.5, 47070]
UP(U)

DN(D)
DN(D)

UP(D)
2021-08-17 03:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[46531.5, 46486.5, 46326.5] [46151.5, 46170, 45911.5]
UP(U)

DN(D)
DN(U)







DN(U)
DN(D)
DN(D)

UP(U)
2021-08-17 19:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[46867, 46348.5, 46001] [46189.5, 45874.5, 45650]
UP(U)
UP(U)
2

TypeError: string indices must be integers